# 텍스트에서 이미지 생성을 위한 stable diffusion 모델 파인튜닝

***
Amazon [SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html)에 오신 것을 환영합니다! JumpStart를 사용하면 SageMaker Studio에서 원클릭으로 또는 [SageMaker JumpStart API](https://sagemaker.readthedocs.io/en/stable/overview.html#use-prebuilt-models-with-sagemaker-jumpstart)를 통해 다양한 머신 러닝 작업을 해결할 수 있습니다. 이 데모 노트북에서는 JumpStart API를 사용하여 최첨단 Stable Diffusion 모델로 텍스트에서 이미지를 생성하는 방법을 보여드립니다. 또한 모델을 사용자의 데이터셋에 맞게 미세 조정하는 방법도 보여드립니다.

Stable Diffusion은 텍스트 프롬프트만으로 사실적인 이미지를 생성할 수 있게 해주는 텍스트-이미지 변환 모델입니다. 확산 모델은 실제 이미지에 추가된 노이즈를 제거하는 방법을 학습함으로써 훈련됩니다. 이 노이즈 제거 과정은 사실적인 이미지를 생성합니다. 이러한 모델은 생성 과정을 텍스트에 조건화함으로써 텍스트만으로도 이미지를 생성할 수 있습니다. 예를 들어, Stable Diffusion은 잠재 확산 모델로, 순수한 노이즈 이미지에서 형태를 인식하고 이러한 형태가 입력 텍스트의 단어와 일치하면 점진적으로 이 형태들을 초점에 맞추는 방식으로 작동합니다.

Stable Diffusion과 같은 대규모 모델을 훈련하고 배포하며 추론을 실행하는 것은 종종 cuda 메모리 부족, 페이로드 크기 제한 초과 등과 같은 문제를 포함하여 어려운 과정입니다. JumpStart는 철저히 테스트된 즉시 사용 가능한 스크립트를 제공함으로써 이 과정을 단순화합니다. 또한 권장 인스턴스 유형, 이미지 생성 과정을 안내하는 매개변수 선택 방법, 프롬프트 엔지니어링 등 프로세스의 각 단계에 대한 지침을 제공합니다. 더욱이, 자체 코드를 작성하지 않고도 JumpStart의 80개 이상의 확산 모델 중 어느 것이든 배포하고 추론을 실행할 수 있습니다.

이 노트북에서는 JumpStart를 사용하여 Stable Diffusion 모델을 사용자의 데이터셋에 맞게 미세 조정하는 방법을 배우게 됩니다. 이는 예술 작품, 로고, 맞춤형 디자인, NFT 등을 만들거나, 반려동물의 AI 이미지나 자신의 아바타와 같은 재미있는 콘텐츠를 생성할 때 유용할 수 있습니다.

모델 라이선스: 이 모델을 사용함으로써, 귀하는 [CreativeML Open RAIL-M++ 라이선스](https://huggingface.co/stabilityai/stable-diffusion-2/blob/main/LICENSE-MODEL)에 동의하게 됩니다.

***

1. [설정](#1.-Set-Up)
2. [사전 훈련된 모델을 커스텀 데이터셋으로 파인 튜닝하기](#2.-Fine-tune-the-pre-trained-model-on-a-custom-dataset)
    * [훈련 아티팩트 검색](#2.1.-Retrieve-Training-Artifacts)
    * [훈련 매개변수 설정](#2.2.-Set-Training-parameters)
    * [훈련 시작](#2.3.-Start-Training)
    * [파인 튜닝된 모델 배포 및 추론 실행](#3.2.-Deploy-and-run-inference-on-the-fine-tuned-model)
3. [결론](#3.-Conclusion)

참고: 이 노트북은 Python 3(Data Science) 커널이 있는 Amazon SageMaker Studio의 ml.t3.medium 인스턴스와 conda_python3 커널이 있는 Amazon SageMaker Notebook 인스턴스에서 테스트되었습니다.

참고: 자신의 데이터셋으로 모델을 미세 조정하려면 계정에서 `ml.g4dn.2xlarge` 인스턴스 유형을 사용할 수 있어야 합니다. 미세 조정된 모델을 배포하려면 `ml.p3.2xlarge` 또는 `ml.g4dn.2xlarge` 인스턴스 유형을 사용할 수 있습니다. 해당 지역에서 `ml.g5.2xlarge`를 사용할 수 있다면, 배포에 해당 인스턴스 유형을 사용하는 것이 좋습니다.

## 1. Set Up

***
노트북을 실행하기 전에 설정을 위한 몇 가지 초기 단계가 필요합니다. 이 노트북은 sagemaker의 최신 버전이 필요합니다.

***

#### 권한 및 환경 변수

***
Amazon SageMaker에서 호스팅하기 위해서는 AWS 서비스 사용을 위한 설정 및 인증이 필요합니다. 여기서는 현재 노트북과 연결된 실행 역할을 SageMaker 접근 권한이 있는 AWS 계정 역할로 사용합니다.

***

In [ ]:
import os
import sagemaker, boto3, json
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

## 2. 사전 훈련된 모델을 커스텀 데이터셋에 파인튜닝하기

---
이전에는 사전 훈련된 모델에서 추론을 실행하는 방법을 살펴보았습니다. 다음으로, 모델을 임의의 클래스 수를 가진 커스텀 데이터셋에 파인튜닝하는 방법에 대해 논의하겠습니다.

모델은 어떤 이미지 데이터셋에도 파인튜닝할 수 있습니다. 훈련 이미지가 5개만 있어도 매우 잘 작동합니다.

파인튜닝 스크립트는 [dreambooth](https://dreambooth.github.io/)의 스크립트를 기반으로 구축되었습니다. 파인튜닝으로 반환된 모델은 추론을 위해 추가로 배포될 수 있습니다. 아래는 훈련 데이터 형식에 대한 지침입니다.

- **입력:** 인스턴스 이미지, `dataset_info.json` 및 (선택 사항) `class_data_dir` 디렉토리를 포함하는 디렉토리.
  - 이미지는 `.png` 또는 `.jpg` 또는 `.jpeg` 형식일 수 있습니다.
  - `dataset_info.json` 파일은 {'instance_prompt':<<instance_prompt>>,'class_prompt':<<class_prompt>>} 형식이어야 합니다.
  - with_prior_preservation = False인 경우, 'class_prompt'를 무시해도 됩니다.
  - `class_data_dir` 디렉토리에는 클래스 이미지가 있어야 합니다. with_prior_preservation = True이고 class_data_dir이 없거나 class_data_dir에 이미 존재하는 이미지가 충분하지 않은 경우, class_prompt로 추가 이미지가 샘플링됩니다.
- **출력:** 추론을 위해 배포할 수 있는 훈련된 모델.

s3 경로는 `s3://bucket_name/input_directory/`와 같아야 합니다. 끝에 `/`가 필요합니다.

다음은 훈련 데이터의 예시 형식입니다.

    input_directory
        |---instance_image_1.png
        |---instance_image_2.png
        |---instance_image_3.png
        |---instance_image_4.png
        |---instance_image_5.png
        |---dataset_info.json
        |---class_data_dir
            |---class_image_1.png
            |---class_image_2.png
            |---class_image_3.png
            |---class_image_4.png

**사전 보존, 인스턴스 프롬프트 및 클래스 프롬프트:** 사전 보존은 훈련하려는 동일한 클래스의 추가 이미지를 사용하는 기술입니다. 예를 들어, 훈련 데이터가 특정 개의 이미지로 구성된 경우, 사전 보존을 통해 일반적인 개의 클래스 이미지를 포함합니다. 특정 개를 훈련하는 동안 다른 개의 이미지를 보여줌으로써 과적합을 방지하려고 합니다. 인스턴스 프롬프트에 있는 특정 개를 나타내는 태그는 클래스 프롬프트에서는 누락됩니다. 예를 들어, 인스턴스 프롬프트는 "a photo of a Doppler dog"이고 클래스 프롬프트는 "a photo of a dog"일 수 있습니다. with_prior_preservation = True로 설정하여 사전 보존을 활성화할 수 있습니다.

개 이미지의 기본 데이터셋을 제공합니다. 이 데이터셋은 클래스 이미지가 없는 단일 개의 이미지(인스턴스 프롬프트에 해당하는 인스턴스 이미지)로 구성됩니다. 기본 데이터셋을 사용하는 경우, 데모 노트북에서 추론을 수행할 때 "a photo of a Doppler dog" 프롬프트를 시도해 보세요.

라이선스: [MIT](https://github.com/marshmellow77/dreambooth-sm/blob/main/LICENSE).

### 2.1. 학습 아티팩트 검색

---
여기서는 학습 도커 컨테이너, 학습 알고리즘 소스 및 사전 학습된 기본 모델을 검색합니다. model_version="*"는 최신 모델을 가져온다는 점에 유의하세요.

---

In [ ]:
from sagemaker import image_uris, model_uris, script_uris

# Currently, not all the stable diffusion models in jumpstart support finetuning. Thus, we manually select a model
# which supports finetuning.
train_model_id, train_model_version, train_scope = (
    "model-txt2img-stabilityai-stable-diffusion-v2-1-base",
    "*",
    "training",
)

# Tested with ml.p3.2xlarge (),ml.g4dn.2xlarge (16GB GPU memory),and ml.g5.2xlarge (24GB GPU memory) instances. Other instances may work as well.
# If ml.g5.2xlarge instance type is available, please change the following instance type to speed up training. Note: If using Workshop Studio accounts,
# leave the default instance below.
training_instance_type = "ml.p3.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)

# Retrieve the training script. This contains all the necessary files including data processing, model training etc.
train_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)

### 2.2. 학습 매개변수 설정

---
이제 필요한 모든 설정을 완료했으므로, 안정적 확산(stable diffusion) 모델을 학습할 준비가 되었습니다. 먼저 [``sageMaker.estimator.Estimator``](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) 객체를 생성해 보겠습니다. 이 estimator는 학습 작업을 시작합니다.

학습을 위해 설정해야 할 매개변수는 두 종류가 있습니다. 첫 번째는 학습 작업을 위한 매개변수입니다. 여기에는 다음이 포함됩니다: (i) 학습 데이터 경로: 입력 데이터가 저장된 S3 폴더입니다. (ii) 출력 경로: 학습 출력이 저장되는 S3 폴더입니다. (iii) 학습 인스턴스 유형: 학습을 실행할 머신의 유형을 나타냅니다. 위에서 올바른 train_image_uri를 가져오기 위해 학습 인스턴스 유형을 정의했습니다.

두 번째 매개변수 세트는 알고리즘별 학습 하이퍼파라미터입니다.

---

In [ ]:
# Sample training data is available in this bucket
training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
training_data_prefix = "training-datasets/dogs_sd_finetuning/"

training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"

output_bucket = sess.default_bucket()
output_prefix = "jumpstart-example-sd-training"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

---
알고리즘별 하이퍼파라미터의 경우, 알고리즘이 기본값과 함께 허용하는 학습 하이퍼파라미터의 Python 딕셔너리를 가져오는 것부터 시작합니다. 이는 나중에 사용자 지정 값으로 재정의할 수 있습니다.

---

In [ ]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)

# [Optional] Override default hyperparameters with custom values
hyperparameters["max_steps"] = "400"
hyperparameters["with_prior_preservation"] = True
print(hyperparameters)

---
`with_prior_preservation=True`로 설정하는 경우, 클래스 이미지를 생성하기 위해 더 많은 메모리가 필요하므로 ml.g5.2xlarge 인스턴스 유형을 사용하세요. 현재 `with_prior_preservation=True`로 설정하면 ml.g4dn.2xlarge 인스턴스 유형에서 학습 시 CUDA 메모리 부족 문제가 발생합니다.

---

#### 원격 훈련 위치에서 로컬로 훈련에 사용된 데이터셋을 다운로드하고 살펴보기
다음 코드 블록을 실행한 후, 현재 위치에 새로 생성된 `training-datasets` 폴더에서 다운로드된 데이터셋을 찾을 수 있습니다.

In [ ]:
s3_resource = boto3.resource('s3')
bucket = s3_resource.Bucket(training_data_bucket) 
for obj in bucket.objects.filter(Prefix = training_data_prefix):
    if not os.path.exists(os.path.dirname(obj.key)):
        os.makedirs(os.path.dirname(obj.key))
    bucket.download_file(obj.key, obj.key) # save to same path

### 2.3. 훈련 시작
---
필요한 모든 자산을 갖춘 estimator 객체를 생성한 다음 훈련 작업을 시작합니다. 기본 데이터셋에서는 10분 미만이 소요됩니다.

---

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-example-{train_model_id}-transfer-learning")

# Create SageMaker Estimator instance
sd_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",  # Entry-point file in source_dir and present in train_source_uri.
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
)

# Launch a SageMaker Training job by passing s3 path of the training data
sd_estimator.fit({"training": training_dataset_s3_path}, logs=True)

### 2.4. 미세 조정된 모델 배포 및 추론 실행

---

훈련된 모델은 그 자체로는 아무것도 하지 않습니다. 이제 모델을 사용하여 추론을 수행하고자 합니다. 이 예제에서는 이미지의 경계 상자를 예측하는 것을 의미합니다. [2. 사전 훈련된 모델에서 추론 실행](#2.-Run-inference-on-the-pre-trained-model)과 동일한 단계를 따릅니다. 먼저 엔드포인트 배포를 위한 jumpstart 아티팩트를 검색합니다. 그러나 base_predictor 대신 미세 조정한 `od_estimator`를 배포합니다.

---

In [ ]:
inference_instance_type = "ml.g4dn.2xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri. This includes scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"jumpstart-example-FT-{train_model_id}-")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = sd_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

다음으로, 파인튜닝된 모델을 쿼리하고, 응답을 파싱하여 생성된 이미지를 표시합니다. 이를 위한 함수들이 아래에 구현되어 있습니다. 엔드포인트의 입력은 json으로 덤프되고 `utf-8` 형식으로 인코딩된 텍스트 문자열입니다. 엔드포인트의 출력은 생성된 텍스트가 포함된 json입니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def query(model_predictor, text):
    """Query the model predictor."""

    encoded_text = text.encode("utf-8")

    query_response = model_predictor.predict(
        encoded_text,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
    )
    return query_response


def parse_response(query_response):
    """Parse response and return generated image and the prompt"""

    response_dict = json.loads(query_response)
    return response_dict["generated_image"], response_dict["prompt"]


def display_img_and_prompt(img, prmpt):
    """Display hallucinated image."""
    plt.figure(figsize=(12, 12))
    plt.imshow(np.array(img))
    plt.axis("off")
    plt.title(prmpt)
    plt.show()

In [ ]:
text = "a photo of a Doppler dog with a hat"
query_response = query(finetuned_predictor, text)
img, prmpt = parse_response(query_response)
display_img_and_prompt(img, prmpt)

[2.4. 지원되는 추론 매개변수](#2.4.-Supported-Inference-parameters)에서 언급된 모든 매개변수는 미세 조정된 모델에서도 지원됩니다. `accept`를 변경하여 [2.5. 압축된 이미지 출력](#2.5.-Compressed-Image-Output)에서와 같이 압축된 이미지 출력을 받을 수도 있습니다.

---
다음으로, 파인튜닝된 모델에 해당하는 엔드포인트를 삭제하겠습니다.

---

In [ ]:
# Delete the SageMaker endpoint
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

## 3. 결론
---

인상적인 이미지 생성은 예술부터 NFT 등 다양한 산업에서 활용될 수 있지만, 오늘날 우리는 AI가 개인화 가능하기를 기대합니다. JumpStart는 사전 훈련된 모델에 미세 조정 기능을 제공하여 최소 다섯 개의 훈련 이미지만으로도 자신의 사용 사례에 맞게 모델을 조정할 수 있습니다. 이는 예술 작품, 로고, 맞춤형 디자인, NFT 등을 만들거나, 반려동물의 AI 이미지나 자신의 아바타와 같은 재미있는 콘텐츠를 생성할 때 유용할 수 있습니다. 이 실습에서는 stable diffusion 텍스트-이미지 생성 모델을 미세 조정하는 방법을 배웠습니다. Stable Diffusion 미세 조정에 대해 더 자세히 알아보려면 [Fine-tune text-to-image Stable Diffusion models with Amazon SageMaker JumpStart](https://aws.amazon.com/blogs/machine-learning/fine-tune-text-to-image-stable-diffusion-models-with-amazon-sagemaker-jumpstart/) 블로그를 확인하시기 바랍니다.